In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Creating Training and Validation data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths for the data
train_data_dir = 'train'
img_size = 224

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,         # Rescale pixel values to the range [0, 1]
    shear_range=0.2,        # Apply shear transformations
    zoom_range=0.2,         # Apply zoom transformations
    horizontal_flip=True,   # Apply horizontal flips
    validation_split=0.2    # Split 20% of the data for validation
)

# Training data
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=64,
    class_mode='categorical',
    subset='training'       # Use a subset of the data for training
)

# Validation data
validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=64,
    class_mode='categorical',
    subset='validation'     # Use a subset of the data for validation
)


## Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Create the CNN model
def build_model(hp):
    model = Sequential()

    # 1. Convolutional Layer
    model.add(Conv2D(hp.Int('conv1_units', min_value=16, max_value=64, step=16), (3, 3), input_shape=(img_size, img_size, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 2. Convolutional Layer
    model.add(Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 3. Convolutional Layer
    model.add(Conv2D(hp.Int('conv3_units', min_value=64, max_value=256, step=64), (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Global Average Pooling Layer
    model.add(GlobalAveragePooling2D())

    # Fully Connected Layer (Dense Layer)
    model.add(Dense(hp.Int('dense_units', min_value=128, max_value=512, step=128), activation='relu'))

    # Output Layer
    model.add(Dense(train_generator.num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

## Keras Tuner

In [ ]:
from kerastuner.tuners import RandomSearch

# Define the RandomSearch tuner using Keras Tuner
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5,
    directory='my_dir', project_name='hyperparameter_tuning_colored')

# Start hyperparameter optimization with Keras Tuner
tuner.search(train_generator, epochs=10, validation_data=validation_generator)

# Display the best hyperparameters
best_hyperparameters = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print("Best Hyperparameters:", best_hyperparameters)